In [32]:
# !pip3 install langchain_community

In [33]:
from langchain_community.llms import Ollama

is_test = False
base_path = "."
ollama_url = "http://localhost:11434"
ollama_model = "llama2"

In [34]:
import os
import dotenv

base_path = "/mnt/f/Progetti/local-llms-analyse"
dotenv.load_dotenv(base_path + "/.env")

ollama_url = os.environ["OLLAMA_URL"]
ollama_model = os.environ["OLLAMA_MODEL"]

print(f"Ollama URL: {ollama_url}")

Ollama URL: http://172.21.98.127:11434


In [35]:
import requests

test_call = requests.get(ollama_url)
print(f"call status: {test_call.status_code}")
print(f"call headers: {test_call.headers}")
print(test_call.content)

call status: 200
call headers: {'Content-Type': 'text/plain; charset=utf-8', 'Date': 'Wed, 13 Mar 2024 22:50:14 GMT', 'Content-Length': '17'}
b'Ollama is running'


In [36]:
llm = Ollama(model=ollama_model, base_url=ollama_url)

In [37]:
test_llm_response = None
if is_test:
    test_llm_response = llm.invoke("The first man on the moon was ...")
test_llm_response

In [38]:
test_categorization_response = None
if is_test:
    test_categorization_response = llm.invoke(
        """Can you add an appropriate category next to each of the following expenses.  
        Respond with a list of categories separated by commas. 
        For example: 
        Spotify AB by Adyen - Entertainment, 
        Beta Boulders Ams Amsterdam Nld - Sports, 
        etc. 
        Consider this expenses: 
        ISS Catering Services De Meern, 
        Vishandel Sier AMSTELVEEN, 
        Ministerie van Justitie en Veiligheid, 
        Etos AMSTERDAM NLD, 
        Bistro Bar Amsterdam
        """
    )

test_categorization_response

### Read transaction data

In [39]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
from pandas import DataFrame

transactions_data: DataFrame = pd.read_csv(base_path + "/transactions_2022_2023.csv")
transactions_data.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [40]:
# Get unique transactions in the Name / Description column
unique_transaction_labels = transactions_data["Name / Description"].unique()
unique_labels_count = len(unique_transaction_labels)
print(f"unique labels: {unique_labels_count}\n{unique_transaction_labels}")

unique labels: 31
['Belastingdienst' 'Tesco Breda' 'Monthly Appartment Rent'
 'Vishandel Sier Amsterdam' 'Selling Paintings' 'Spotify Ab By Adyen'
 'Tk Maxx Amsterdam Da' 'Consulting' 'Aidsfonds' 'Tls Bv Inz Ov-Chipkaart'
 'Etos Amsterdam' 'Beta Boulders Ams Amsterdam' 'Salary'
 'Bouldermuur Bv Amsterdam' 'Birtat Restaurant Amsterdam' 'Freelancing'
 'Tikkie' 'Blogging' 'Taxi Utrecht' 'Apple Services' 'Amazon Lux'
 'Classpass* Monthly' 'Audible Uk AdblCo/Pymt Gbr' 'Twitch Lux'
 'PayPal ltd' 'NTT Data' 'ATM Milano' 'Tenoha Milano'
 'Librerie La Feltrinelli' 'Muji' 'LEGO Store Babila']


### Categorise bank transactions with Llama2

In [41]:
chunk_size = 25
labels_groups = [list(unique_transaction_labels[i: i + chunk_size]) for i in
                 range(0, len(unique_transaction_labels), chunk_size)]
labels_groups

[['Belastingdienst',
  'Tesco Breda',
  'Monthly Appartment Rent',
  'Vishandel Sier Amsterdam',
  'Selling Paintings',
  'Spotify Ab By Adyen',
  'Tk Maxx Amsterdam Da',
  'Consulting',
  'Aidsfonds',
  'Tls Bv Inz Ov-Chipkaart',
  'Etos Amsterdam',
  'Beta Boulders Ams Amsterdam',
  'Salary',
  'Bouldermuur Bv Amsterdam',
  'Birtat Restaurant Amsterdam',
  'Freelancing',
  'Tikkie',
  'Blogging',
  'Taxi Utrecht',
  'Apple Services',
  'Amazon Lux',
  'Classpass* Monthly',
  'Audible Uk AdblCo/Pymt Gbr',
  'Twitch Lux',
  'PayPal ltd'],
 ['NTT Data',
  'ATM Milano',
  'Tenoha Milano',
  'Librerie La Feltrinelli',
  'Muji',
  'LEGO Store Babila']]

In [42]:
def response_check(value: list[str]):
    for item in value:
        assert "-" in item, "String does not contain hyphen."


In [43]:
import re


def categorize_transactions(transaction_names: list[str], llm: Ollama):
    raw_response = llm.invoke(
        "Avoiding any explanation, add an appropriate category to each of the following expenses: " + ",".join(transaction_names))
    # print(f"raw response: {response}") # DEBUG
    response = raw_response.split('\n')

    # Keep only the lines in between blank lines (removing the explanation lines at the beginning and end of the response)
    blank_indexes = [index for index in range(0, len(response)) if response[index] == '']
    if len(blank_indexes) == 1:
        response = response[(blank_indexes[0] + 1):]
    else:
        response = response[(blank_indexes[0] + 1): blank_indexes[1]]

    # Print response and validate if it is in the correct format
    response = [re.sub(r"\d+\. ", "", r) for r in response if len(r) > 0]
    response_check(response)
    
    if len(transaction_names) != len(response):
        print(f"mismatch in response number!\ngiven labels: {transaction_names}\nreceived response: {raw_response}\n")
    # Put in dataframe
    categories_df = pd.DataFrame({"Transaction vs category": response})
    categories_df[["Transaction", "Category"]] = categories_df["Transaction vs category"].str.split(' - ', expand=True)
    categories_df.drop(["Transaction vs category"], axis=1, inplace=True)

    return categories_df

In [44]:
# Test out the function
test_categorization_function_response = None
if is_test:
    test_categorization_function_response = categorize_transactions(
        ['ISS Catering Services De Meern', 'Vishandel Sier AMSTELVEEN', 'Etos AMSTERDAM NLD', 'Bistro Bar Amsterdam'],
        llm
    )
test_categorization_function_response

In [45]:
categories_dataframe = DataFrame()
max_tries = 3

for labels in labels_groups:
    tries = 0
    mapped_categories = None
    while tries < max_tries and mapped_categories is None:
        try:
            mapped_categories = categorize_transactions(labels, llm)
        except Exception as e:
            if tries < max_tries:
                tries += 1
            else:
                raise Exception(f"Failed categorizing transactions for {labels}") from e
        categories_dataframe = pd.concat([categories_dataframe, mapped_categories], ignore_index=True)

categories_dataframe

,Transaction,Category
0,NTT Data,Technology
1,ATM Milano,Finance
2,Tenoha Milano,Fashion
3,Librerie La Feltrinelli,Books
4,Muji,Home & Decor
5,LEGO Store Babila,Toys


In [46]:
# Drop NA values
categories_dataframe = categories_dataframe.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_dataframe.loc[categories_dataframe['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_dataframe.loc[categories_dataframe['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_dataframe.loc[categories_dataframe['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_dataframe.loc[categories_dataframe['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_dataframe.loc[categories_dataframe['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_dataframe.loc[categories_dataframe['Category'].str.contains("Travel"), 'Category'] = "Travel"
categories_dataframe

,Transaction,Category
0,NTT Data,Technology
1,ATM Milano,Finance
2,Tenoha Milano,Fashion
3,Librerie La Feltrinelli,Books
4,Muji,Home & Decor
5,LEGO Store Babila,Toys


In [47]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
transactions_data = pd.read_csv(base_path + "/transactions_2022_2023.csv")
transactions_data.loc[transactions_data['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify AB By Adyen"
transactions_data["ND_JOIN"] = transactions_data["Name / Description"].str.upper()
categories_dataframe["T_JOIN"] = categories_dataframe["Transaction"].str.upper()
transactions_data = pd.merge(transactions_data, categories_dataframe, left_on="ND_JOIN", right_on="T_JOIN", how="left")
transactions_data.drop(["ND_JOIN", "T_JOIN", "Transaction"], axis=1, inplace=True)
transactions_data

,Date,Name / Description,Expense/Income,Amount (EUR),Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN
5,2023-12-29,Spotify AB By Adyen,Expense,12.19,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN
7,2023-12-22,Consulting,Income,541.57,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN


In [48]:
transactions_data.to_csv(base_path + "/transactions_2022_2023_categorized.csv", index=False)